In [47]:
pip install optuna

Note: you may need to restart the kernel to use updated packages.


In [48]:
optuna.logging.set_verbosity(optuna.logging.WARNING) #Eliminar prints ruidosos durante el entrenamiento

In [57]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve
import xgboost as xgb
import optuna

#Cargar dataset
file_path = "dataset_Caso_1.csv"
df = pd.read_csv(file_path, delimiter=",")

#Convertir columnas x1 y x2 a números asegurando que queden valores numéricos
df["x1"] = pd.to_numeric(df["x1"], errors="coerce")
df["x2"] = pd.to_numeric(df["x2"], errors="coerce")

#Verificar si hay valores nulos tras la conversión y eliminarlos
df.dropna(subset=["x1","x2"], inplace=True)

#Convertir variables categóricas (x3 y x4) a numéricas usando LabelEncoder
label_encoder_x3 = LabelEncoder()
label_encoder_x4 = LabelEncoder()

df['x3'] = label_encoder_x3.fit_transform(df['x3'])
df['x4'] = label_encoder_x4.fit_transform(df['x4'])

#Eliminar filas con valores NaN
df.dropna(inplace=True)

#Separar las características(X) y la variable objetivo(y)
X = df.drop(columns=['target'])
y = df['target'].astype(int)  #Asegurar que target sea numérico

#Calcular scale_pos_weight (para datos desbalanceados) #Hay considerablemente mayor cantidad de 0 que de 1
num_neg = (y == 0).sum() #Contar clase 0
num_pos = (y == 1).sum() #Contar clase 1
scale_pos_weight = num_neg / num_pos

#Dividir el dataset en 85% entrenamiento y 15% prueba
X_train, X_test, y_train, y_test = train_test_split( 
    X, y, test_size=0.15, random_state=42, stratify=y #Usar stratify para que ambas clases tengan la misma proporción
)


In [58]:
#Función objetivo para Optuna (Configuración hiperparámetros)
def objective(trial):
    params = {
        'eval_metric': 'auc', 
        'max_depth': trial.suggest_int('max_depth', 4, 9),
        'learning_rate': trial.suggest_float('learning_rate', 0.0001, 0.3, log=True),
        'subsample': trial.suggest_float('subsample', 0.5, 0.6),  
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.6, 1.0),
        'lambda': trial.suggest_float('lambda', 1e-3, 10, log=True),
        'alpha': trial.suggest_float('alpha', 1e-3, 10, log=True),
        'scale_pos_weight': scale_pos_weight,
        'n_estimators': trial.suggest_int('n_estimators', 10, 400),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 6), 
        'gamma': trial.suggest_float('gamma', 0, 5),  #Evita overfitting
        'random_state': 42 }

    #Definir el modelo base de XGBoost
    model = xgb.XGBClassifier(**params,)
    model.fit(X_train,y_train)
    y_test_pred = model.predict(X_test)
    f1 = f1_score(y_test, y_test_pred)    
    return f1

#Maximizar F1 con optuna
study = optuna.create_study(direction="maximize") 
study.optimize(objective, n_trials=100, show_progress_bar=True) #Barra de progreso hasta encontrar el mejor valor para F1

#Obtener mejores hiperparámetros
best_params = study.best_params
print("Mejores hiperparámetros encontrados:",best_params)

#Entrenar el modelo con los mejores parámetros encontrados
mejor_modelo = xgb.XGBClassifier(**best_params, eval_metric='auc', random_state=42, scale_pos_weight=scale_pos_weight)
mejor_modelo.fit(X_train, y_train)

#Predicciones en Train y Test
y_train_pred = mejor_modelo.predict(X_train)
y_test_pred = mejor_modelo.predict(X_test)

#Calcular las probabilidades de la clase 1
y_test_pred_proba = mejor_modelo.predict_proba(X_test)[:, 1]

#Calcular métricas
auc_train = roc_auc_score(y_train, y_train_pred)
auc_test = roc_auc_score(y_test, y_test_pred)
f1 = f1_score(y_test, y_test_pred)

#Mostrar resultados
print(f'\nAUC Train: {auc_train:.4f}')
print(f'AUC Test: {auc_test:.4f}')
print(f'F1-Score: {f1:.4f}')

  0%|          | 0/100 [00:00<?, ?it/s]

Mejores hiperparámetros encontrados: {'max_depth': 6, 'learning_rate': 0.00102084373660682, 'subsample': 0.5140277784709432, 'colsample_bytree': 0.953923815878277, 'lambda': 0.7769438351764247, 'alpha': 0.010118784188155818, 'n_estimators': 191, 'min_child_weight': 4, 'gamma': 3.9474720039996924}

AUC Train: 0.9384
AUC Test: 0.6633
F1-Score: 0.3333
